<a href="https://colab.research.google.com/github/MeusEstudos/RedesNeuraisFFNN/blob/main/Engenharia_dos_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# REDE NEURAL COM PREDIÇÕES DE VOLUME ÚTIL DO RESERVATÓRIO DE FUNIL (RJ) BASEADO EM VARIÁVEIS CLIMÁTICAS

###### Tratamento e processamento de dados das variáveis climáticas e do volume útil do reservatório

## Introdução

O objetivo da pesquisa foi verificar como as variáveis climáticas podem impactar no volume útil de um reservatório, utilizando Redes Neurais.

- Os dados climáticos são advindos do [INMET](https://bdmep.inmet.gov.br/) de acordo com a estação mais próxima da coordenada do reservatório escolhido, realizado essa localização pelo recurso do [Meteostat](https://github.com/meteostat/meteostat-python).

- O reservetório escolhido foi de acordo com a quantidade de dados disponíveis em uma das estações meteorológicas mais próxima dos principais reservatórios que abastecem o Rio de Janeiro, coletado pelo  [Sistema de Acompanhamento de Reservatórios (SAR)](https://www.ana.gov.br/sar0/MedicaoSin?dropDownListEstados=20&dropDownListReservatorios=19093&dataInicial=01%2F01%2F2012&dataFinal=23%2F02%2F2022&button=Buscar) mantido pela Agência Nacional de Águas e Saneamento Básico (ANA).

- A técnica de aprendizagem de máquinda adotada como foco principal de estudos foi a [Feed-forward Neural Network (FFNN)](https://stringfixer.com/pt/Feedforward_neural_networks), onde de acordo com o [Zell, Andreas (1994)](https://towardsdatascience.com/deep-learning-feedforward-neural-network-26a6705dbdc7) trata-se de "uma rede neural artificial em que as conexões entre os nós não formam um ciclo. Como tal, é diferente de seu descendente: redes neurais recorrentes". E segundo [NAGASELVI e DEEPA (2015)](http://jset.sasapublications.com/wp-content/uploads/2017/10/6702690.pdf), o tempo é uma variável amplamente utilizada em redes neurais onde em sua pesquisa ao utilizar a Rede Neural FeedForward eles obtiveram resultados satisfatórios com dados meteorológicos, mesmo com um dataset pequeno e com dados ausentes. No [dicionário técnico de tradução](https://www.dicionariotecnico.com/traducao.php?termo=Feed-forward+Neural+Network) do ingês para o português, o Feed-forward Neural Network da sigla em inglês FFNN, se traduz em: 
  - Rede Neural de Alimentação Direta, ou;
  - Rede Neural de Alimentação Antecipada.

## 1 - Importação das bibliotecas

In [1]:
# Importação dos arquivos do seu Google Drive pelo Colab - https://colab.research.google.com/notebooks/io.ipynb 
from google.colab import drive
drive.mount('/content/drive')

# Processamento/manipulação dos dados https://pandas.pydata.org/
import pandas as pd

# Dependências do Meteostat https://github.com/meteostat
!pip install meteostat
from datetime import datetime
from meteostat import Stations

# Gráficos https://matplotlib.org/
import matplotlib.pyplot as plt
!pip install seaborn
!pip install plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Mounted at /content/drive


## 2 - Estações meteorológicas próximas aos principais reservatórios do Rio

- **Estação meteorológica: Resende | Id: 83738 | Reservatório: FUNIL | Distância: 12km**
- Estação meteorológica: Resende | Id: 86874 | Reservatório: FUNIL | Distância: 15km
- Estação meteorológica: Sao Luis Do Paraitinga / Rio Afonsos | Id: 86912 | Reservatório: PARAIBUNA | Distância: 28km

In [2]:
# criação de um dicionário que guarda o nome de cada reservatório e de cada coordenada correspondente a ele
dic_coordenadas_reservatorios = {"FUNIL":  (-22.5311, -44.5681), "JAGUARI": (-23.1956, -46.0075), "PARAIBUNA": (-23.415, -45.6025), "SANTA BRANCA": (-23.3743, -45.8727)}

# criação de um dicionário vazio que guardará cada nome de dados da estação mais próxima relativa a um reservatório
dic_reservatorios_data_estacoes = {} 

# iteração sobre cada nome de reservatório e cada coordenada guardando os nomes em variáveis (chave e valor)
for (chave, valor) in dic_coordenadas_reservatorios.items():

    # separação de cada coordenada do reservatório em variáveis diferentes
    lat_reservatorio = valor[0]
    long_reservatorio = valor[1]

    # criação do objeto Stations para procura e retorno de uma ou mais estações
    stations = Stations()

    # captura de 6 estações perto do reservatório, passando as coordenadas dele
    stations = stations.nearby(lat_reservatorio, long_reservatorio).fetch(6)

    for index, linha in stations.iterrows():

        # extração do id, nome da estação e distância de no máximo 32km em relação ao reservatório
        id, nome, dist = linha.wmo, linha["name"], linha['distance']
        if (dist < 32000.0):
            print('Estação meteorológica: ' + nome + ' | Id: ' + id + ' | Reservatório: ' + chave + ' | Distância: ' + str(round(dist / 1000.0)) + 'km')
        else:
            pass

Estação meteorológica: Resende | Id: 83738 | Reservatório: FUNIL | Distância: 12km
Estação meteorológica: Resende | Id: 86874 | Reservatório: FUNIL | Distância: 15km
Estação meteorológica: Sao Luis Do Paraitinga / Rio Afonsos | Id: 86912 | Reservatório: PARAIBUNA | Distância: 28km


ANTES DE DETERMINAR A DISTÂNCIA MÁXIMA DA ESTAÇÃO EM RELAÇÃO AO RESERVATÓRIO

A estação meteorológica que tinha tido menos perda após o tratamentos dos dados, de apenas 2%, foi a de São Lourenço (Id: 83736) do reservatório de FUNIL. Mas esta estação apresenta uma distância de 67km, muito além de alguma variável climática registrada nessa estação afetar algo no reservatório. Onde segundo [Ray, Ram. (2017)](https://www.researchgate.net/post/Data-and-weather-station-distance/594d2b50217e20b58d25659f/citation/download) em sua resposta sobre "*Data and weather station distance*" foi de considerar 32km.

## 3 - Estação meteorológica de Resende [83738] - (RJ)

- **df_estacao_Resende_diaria**
  - Período: 17/09/1983 - 18/03/2022
  - Quantidade de dados: 14.062

- **df_estacao_Resende_diaria_convertida**
  - Período: 17/09/1983 - 04/12/2017
  - Quantidade de dados: 6.027

- **Perda de dados**
  - 57%

### A - Informações detalhada sobre a coleta dos dados no INMET

Extração pelo site do [INMET](https://bdmep.inmet.gov.br/)

*Data Inicio: 17/09/1983 | Data Fim: 18/03/2022*

---

**Estações próximas aos reservatórios detectada pelo METEOSTAT:**

Resende foi a estação disponível no INMET mais próxima de um dos quatro principais reservatórios que abastece o Rio:
- Estação meteorológica: Resende | Id: 83738 | Reservatório: FUNIL

**Estações encontradas no INMET:**

- **RESENDE [83738] - (RJ)**
  - Codigo Estacao: 83738
  - Latitude: -22.45111111
  - Longitude: -44.44472221
  - Altitude: 439.9
  - Situacao: Operante
  - Data Inicial: 17/09/1983
  - Data Final: 18/03/2022
  - Periodicidade da Medicao: Diária

---

**Tipo de Dados: Diário | Tipo de Estação: Convencional**

**Variáveis:**

- EVAPORACAO DO PICHE, DIARIA
- INSOLACAO TOTAL, DIARIO
- PRECIPITACAO TOTAL, DIARIO
- TEMPERATURA MAXIMA, DIARIA
- TEMPERATURA MEDIA COMPENSADA, DIARIA
- TEMPERATURA MINIMA, DIARIA
- UMIDADE RELATIVA DO AR, MEDIA DIARIA
- VENTO, VELOCIDADE MEDIA DIARIA

### B - Análise e tratamento dos dados

#### B1 - Tipo de Dados: Diário | Tipo de Estação: Convencional

In [3]:
# lendo o arquivo
df_estacao_Resende_diaria = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TCC_PUC-Minas/Dados/Estacao_Meteorologica/RESENDE_diaria.csv", sep=";") 
df_estacao_Resende_diaria

,Data Medicao,"EVAPORACAO DO PICHE, DIARIA(mm)","INSOLACAO TOTAL, DIARIO(h)","PRECIPITACAO TOTAL, DIARIO(mm)","TEMPERATURA MEDIA COMPENSADA, DIARIA(°C)","UMIDADE RELATIVA DO AR, MEDIA DIARIA(%)","VENTO, VELOCIDADE MEDIA DIARIA(m/s)"
0,17/09/1983,1.5,0.0,0.7,18.28,81.50,1.566.667
1,18/09/1983,1.2,0.0,30.5,18.94,90.25,.533333
2,19/09/1983,0.7,0.0,18.9,18.78,96.00,2
3,20/09/1983,0.3,0.0,15.8,15.96,80.50,2.9
4,21/09/1983,1.5,0.0,0.2,15.30,84.25,2.633.333
...,...,...,...,...,...,...,...
14058,14/03/2022,NaN,NaN,2.8,NaN,NaN,NaN
14059,15/03/2022,NaN,NaN,0.6,NaN,NaN,NaN
14060,16/03/2022,NaN,NaN,20.2,NaN,NaN,NaN
14061,17/03/2022,NaN,NaN,0.0,NaN,NaN,NaN


In [29]:
# detalhar dados ausentes :: código disponibilizado por OLIVEIRA (2019) no Kaggle => https://www.kaggle.com/code/samukaunt/titanic-passo-a-passo-com-8-modelos-ml-pt-br/notebook 

# soma a quantidade de linhas nulas de cada coluna, transformando a quantidade de cada coluna em linhas de uma tabela nova onde cada index corresponde aos nomes antigos de cada coluna somada, em ordem decrescente 
total = df_estacao_Resende_diaria.isnull().sum().sort_values(ascending=False)

# realiza a mesma operação acima e divide o resultado pelo número total de itens de cada coluna antiga, e depois multiplica por 100 para achar a porcentagem de quantidade de itens nulos vs o total   
percent_1 = df_estacao_Resende_diaria.isnull().sum()/df_estacao_Resende_diaria.isnull().count()*100

# arredonda cada porcentagem para uma casa decimal e traz as porcentagens em ordem decrescente
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)

# concatena as duas colunas geradas acima em um novo dataframe, concatenando pelas colunas (usando o axis=1) e com os nomes das colunas novas como 'Total' e '%'
df_estacao_Resende_diaria_detalhada = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])

# mostra as 9 primeiras linhas do dataframe gerado
df_estacao_Resende_diaria_detalhada.head(7)

,Total,%
"VENTO, VELOCIDADE MEDIA DIARIA(m/s)",6395,45.5
"UMIDADE RELATIVA DO AR, MEDIA DIARIA(%)",5629,40.0
"TEMPERATURA MEDIA COMPENSADA, DIARIA(°C)",5021,35.7
"PRECIPITACAO TOTAL, DIARIO(mm)",3751,26.7
"EVAPORACAO DO PICHE, DIARIA(mm)",2917,20.7
"INSOLACAO TOTAL, DIARIO(h)",2681,19.1
Data Medicao,0,0.0


In [5]:
# renomear colunas
df_estacao_Resende_diaria_renomeada = df_estacao_Resende_diaria.rename(columns={'Data Medicao':'data', 'INSOLACAO TOTAL, DIARIO(h)':'inso', 'EVAPORACAO DO PICHE, DIARIA(mm)':'evap', 'PRECIPITACAO TOTAL, DIARIO(mm)':'prec', 'TEMPERATURA MEDIA COMPENSADA, DIARIA(°C)':'temp', 'UMIDADE RELATIVA DO AR, MEDIA DIARIA(%)':'umid', 'VENTO, VELOCIDADE MEDIA DIARIA(m/s)':'vent'})
df_estacao_Resende_diaria_renomeada

,data,evap,inso,prec,temp,umid,vent
0,17/09/1983,1.5,0.0,0.7,18.28,81.50,1.566.667
1,18/09/1983,1.2,0.0,30.5,18.94,90.25,.533333
2,19/09/1983,0.7,0.0,18.9,18.78,96.00,2
3,20/09/1983,0.3,0.0,15.8,15.96,80.50,2.9
4,21/09/1983,1.5,0.0,0.2,15.30,84.25,2.633.333
...,...,...,...,...,...,...,...
14058,14/03/2022,NaN,NaN,2.8,NaN,NaN,NaN
14059,15/03/2022,NaN,NaN,0.6,NaN,NaN,NaN
14060,16/03/2022,NaN,NaN,20.2,NaN,NaN,NaN
14061,17/03/2022,NaN,NaN,0.0,NaN,NaN,NaN


In [6]:
# retirar os dados faltantes 
df_estacao_Resende_diaria_faltante = df_estacao_Resende_diaria_renomeada.dropna()
df_estacao_Resende_diaria_faltante

,data,evap,inso,prec,temp,umid,vent
0,17/09/1983,1.5,0.0,0.7,18.28,81.50,1.566.667
1,18/09/1983,1.2,0.0,30.5,18.94,90.25,.533333
2,19/09/1983,0.7,0.0,18.9,18.78,96.00,2
3,20/09/1983,0.3,0.0,15.8,15.96,80.50,2.9
4,21/09/1983,1.5,0.0,0.2,15.30,84.25,2.633.333
...,...,...,...,...,...,...,...
12399,28/08/2017,5.4,10.6,0.0,18.54,71.50,154.332
12432,30/09/2017,4.3,0.0,23.8,19.12,97.25,0
12477,14/11/2017,8.9,12.2,0.0,22.06,55.00,0
12480,17/11/2017,9.5,9.0,0.0,24.16,68.75,0


In [7]:
# correção do index gerado pelo dropna
df_estacao_Resende_diaria_corrigida = df_estacao_Resende_diaria_faltante.reset_index(drop=True)
df_estacao_Resende_diaria_corrigida

,data,evap,inso,prec,temp,umid,vent
0,17/09/1983,1.5,0.0,0.7,18.28,81.50,1.566.667
1,18/09/1983,1.2,0.0,30.5,18.94,90.25,.533333
2,19/09/1983,0.7,0.0,18.9,18.78,96.00,2
3,20/09/1983,0.3,0.0,15.8,15.96,80.50,2.9
4,21/09/1983,1.5,0.0,0.2,15.30,84.25,2.633.333
...,...,...,...,...,...,...,...
6023,28/08/2017,5.4,10.6,0.0,18.54,71.50,154.332
6024,30/09/2017,4.3,0.0,23.8,19.12,97.25,0
6025,14/11/2017,8.9,12.2,0.0,22.06,55.00,0
6026,17/11/2017,9.5,9.0,0.0,24.16,68.75,0


In [8]:
# visualizar os tipos de dados
df_estacao_Resende_diaria_corrigida.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6028 entries, 0 to 6027
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   data    6028 non-null   object 
 1   evap    6028 non-null   float64
 2   inso    6028 non-null   float64
 3   prec    6028 non-null   float64
 4   temp    6028 non-null   float64
 5   umid    6028 non-null   float64
 6   vent    6028 non-null   object 
dtypes: float64(5), object(2)
memory usage: 329.8+ KB


In [9]:
# convertendo os tipos de dados
df_estacao_Resende_diaria_convertida = df_estacao_Resende_diaria_corrigida.assign(data=pd.to_datetime(df_estacao_Resende_diaria_corrigida['data']), vent=df_estacao_Resende_diaria_corrigida['vent'].str.replace('.', '').astype(float))
df_estacao_Resende_diaria_convertida.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6028 entries, 0 to 6027
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   data    6028 non-null   datetime64[ns]
 1   evap    6028 non-null   float64       
 2   inso    6028 non-null   float64       
 3   prec    6028 non-null   float64       
 4   temp    6028 non-null   float64       
 5   umid    6028 non-null   float64       
 6   vent    6028 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 329.8 KB


#### B2 - Cálculo da perda de dados após o tratamento

In [10]:
# Estação de Resende = Tipo de Dados: Diário | Tipo de Estação: Convencional
df_estacao_Resende_diaria_pct_perda = 100.0 - (len(df_estacao_Resende_diaria_convertida.index) * 100.0 / len(df_estacao_Resende_diaria.index))
print("Quantidade de perda de dados da estação de Resende com o tipo de dados diário: " + str(round(df_estacao_Resende_diaria_pct_perda)) + "%")

Quantidade de perda de dados da estação de Resende com o tipo de dados diário: 57%


## 4 - Reservatório de Funil (RJ)

- **df_reservatorio_Funil_diario**
  - Período: 01/01/1993 - 05/12/2017
  - Quantidade de dados: 9.104

- **df_reservatorio_Funil_diario_convertido**
  - Período: 01/01/1993 - 05/12/2017
  - Quantidade de dados: 9.104

- **Perda de dados**
  - 0%

### A - Informações detalhada sobre a coleta dos dados no SAR

Extração pelo site do [SAR](https://www.ana.gov.br/sar0/MedicaoSin?dropDownListEstados=20&dropDownListReservatorios=19093&dataInicial=17%2F09%2F1983&dataFinal=04%2F12%2F2017&button=Buscar)

*Data Inicio: 01/01/1993| Data Fim: 05/12/2017*
 
---

**Reservatório próximo a estação detectada pelo METEOSTAT:**

Resende foi a estação disponível no INMET mais próxima do reservatório de Funil:
- Estação meteorológica: Resende | Id: 83738 | Reservatório: FUNIL

---

**Tipo de Dados: Diário** 

**Variáveis:**

- Código do Reservatório
- Reservatório
- Cota (m)
- Afluência (m³/s)
- Defluência (m³/s)
- Vazão Vertida (m³/s)
- Vazão Turbinada (m³/s)
- Vazão Natural (m³/s)
- Volume Útil (%)
- Vazão Incremental (m³/s)
- Data da Medição

### B - Análise e tratamento dos dados

In [11]:
# lendo o arquivo
df_reservatorio_Funil_diario = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TCC_PUC-Minas/Dados/Usina_Hidreletrica/FUNIL_diario.csv", encoding="ISO-8859-1", sep=";") 
df_reservatorio_Funil_diario

,Código,Nome,Cota (m),Afluência (m³/s),Defluência (m³/s),Vazão Vertida (m³/s),Vazão Turbinada (m³/s),Vazão Natural (m³/s),Volume Útil (%),Vazão Incremental (m³/s),Data da Medição
0,19093,FUNIL,"461,14","151,84",195,0,195.0,"149,4","68,49",NaN,01/01/1993
1,19093,FUNIL,"461,02","149,92",197,0,197.0,"148,3","67,83",NaN,02/01/1993
2,19093,FUNIL,"460,99","184,37",196,0,196.0,155,"67,66",NaN,03/01/1993
3,19093,FUNIL,"460,95","180,86",196,0,196.0,"160,9","67,5",NaN,04/01/1993
4,19093,FUNIL,"460,99","193,14",178,0,178.0,"171,5","67,66",NaN,05/01/1993
...,...,...,...,...,...,...,...,...,...,...,...
9100,19093,FUNIL,"453,52","149,53",164,0,164.0,"142,11","31,8","80,27",01/12/2017
9101,19093,FUNIL,"453,46","133,64",151,0,151.0,"126,76","31,55","64,39",02/12/2017
9102,19093,FUNIL,"453,38","129,85",153,0,153.0,"115,51","31,22","59,61",03/12/2017
9103,19093,FUNIL,"453,25","127,38",165,0,165.0,"108,95","30,68","56,65",04/12/2017


In [12]:
# detalhar dados ausentes
total = df_reservatorio_Funil_diario.isnull().sum().sort_values(ascending=False)
percent_1 = df_reservatorio_Funil_diario.isnull().sum()/df_reservatorio_Funil_diario.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
df_reservatorio_Funil_diario_detalhado = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
df_reservatorio_Funil_diario_detalhado.head(11)

,Total,%
Vazão Incremental (m³/s),4803,52.8
Vazão Turbinada (m³/s),2902,31.9
Cota (m),761,8.4
Código,0,0.0
Nome,0,0.0
Afluência (m³/s),0,0.0
Defluência (m³/s),0,0.0
Vazão Vertida (m³/s),0,0.0
Vazão Natural (m³/s),0,0.0
Volume Útil (%),0,0.0


In [13]:
# renomear coluna para unir dataframes
df_reservatorio_Funil_diario_renomeado = df_reservatorio_Funil_diario.rename(columns={'Data da Medição': 'data', 'Volume Útil (%)': 'volu'})
df_reservatorio_Funil_diario_renomeado

,Código,Nome,Cota (m),Afluência (m³/s),Defluência (m³/s),Vazão Vertida (m³/s),Vazão Turbinada (m³/s),Vazão Natural (m³/s),volu,Vazão Incremental (m³/s),data
0,19093,FUNIL,"461,14","151,84",195,0,195.0,"149,4","68,49",NaN,01/01/1993
1,19093,FUNIL,"461,02","149,92",197,0,197.0,"148,3","67,83",NaN,02/01/1993
2,19093,FUNIL,"460,99","184,37",196,0,196.0,155,"67,66",NaN,03/01/1993
3,19093,FUNIL,"460,95","180,86",196,0,196.0,"160,9","67,5",NaN,04/01/1993
4,19093,FUNIL,"460,99","193,14",178,0,178.0,"171,5","67,66",NaN,05/01/1993
...,...,...,...,...,...,...,...,...,...,...,...
9100,19093,FUNIL,"453,52","149,53",164,0,164.0,"142,11","31,8","80,27",01/12/2017
9101,19093,FUNIL,"453,46","133,64",151,0,151.0,"126,76","31,55","64,39",02/12/2017
9102,19093,FUNIL,"453,38","129,85",153,0,153.0,"115,51","31,22","59,61",03/12/2017
9103,19093,FUNIL,"453,25","127,38",165,0,165.0,"108,95","30,68","56,65",04/12/2017


In [14]:
# remover a coluna 
df_reservatorio_Funil_diario_removido = df_reservatorio_Funil_diario_renomeado.drop(['Código','Nome','Cota (m)','Afluência (m³/s)','Defluência (m³/s)','Vazão Vertida (m³/s)','Vazão Turbinada (m³/s)','Vazão Natural (m³/s)','Vazão Incremental (m³/s)'], axis=1)
df_reservatorio_Funil_diario_removido

,volu,data
0,"68,49",01/01/1993
1,"67,83",02/01/1993
2,"67,66",03/01/1993
3,"67,5",04/01/1993
4,"67,66",05/01/1993
...,...,...
9100,"31,8",01/12/2017
9101,"31,55",02/12/2017
9102,"31,22",03/12/2017
9103,"30,68",04/12/2017


In [15]:
# visualizar os tipos de dados
df_reservatorio_Funil_diario_removido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9105 entries, 0 to 9104
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   volu    9105 non-null   object
 1   data    9105 non-null   object
dtypes: object(2)
memory usage: 142.4+ KB


In [16]:
# convertendo os tipos de dados
df_reservatorio_Funil_diario_convertido = df_reservatorio_Funil_diario_removido.assign(data=pd.to_datetime(df_reservatorio_Funil_diario_removido['data']), volu=df_reservatorio_Funil_diario_removido['volu'].str.replace(',', '.').astype(float))
df_reservatorio_Funil_diario_convertido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9105 entries, 0 to 9104
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   volu    9105 non-null   float64       
 1   data    9105 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 142.4 KB


## 5 - Unindo dataframes da estação meteorológica de Resende (RJ) e do reservatório de Funil (RJ)

- **df_estacao_Resende_diaria**
  - Período: 17/09/1983 - 18/03/2022
  - Quantidade de dados: 14.062

- **df_estacao_Resende_diaria_convertida**
  - Período: 17/09/1983 - 04/12/2017
  - Quantidade de dados: 6.027
  - Perda de dados: 57% [df_estacao_Resende_diaria - df_estacao_Resende_diaria_convertida]

- **df_reservatorio_Funil_diario_convertido**
  - Período: 01/01/1993 - 05/12/2017
  - Quantidade de dados: 9.104
  - Perda de dados: 0%

- **df_estacao_Resende_E_df_reservatorio_Funil**
  - Período: 04/02/1993 - 17/11/2017
  - Quantidade de dados: 5.891

- **Perda de dados**
  - 58% [df_estacao_Resende_diaria - df_estacao_Resende_E_df_reservatorio_Funil]

### A - Junção e exportação dos dados

In [17]:
df_estacao_Resende_diaria_convertida

,data,evap,inso,prec,temp,umid,vent
0,1983-09-17,1.5,0.0,0.7,18.28,81.50,1566667.0
1,1983-09-18,1.2,0.0,30.5,18.94,90.25,533333.0
2,1983-09-19,0.7,0.0,18.9,18.78,96.00,2.0
3,1983-09-20,0.3,0.0,15.8,15.96,80.50,29.0
4,1983-09-21,1.5,0.0,0.2,15.30,84.25,2633333.0
...,...,...,...,...,...,...,...
6023,2017-08-28,5.4,10.6,0.0,18.54,71.50,154332.0
6024,2017-09-30,4.3,0.0,23.8,19.12,97.25,0.0
6025,2017-11-14,8.9,12.2,0.0,22.06,55.00,0.0
6026,2017-11-17,9.5,9.0,0.0,24.16,68.75,0.0


In [18]:
df_reservatorio_Funil_diario_convertido

,volu,data
0,68.49,1993-01-01
1,67.83,1993-02-01
2,67.66,1993-03-01
3,67.50,1993-04-01
4,67.66,1993-05-01
...,...,...
9100,31.80,2017-01-12
9101,31.55,2017-02-12
9102,31.22,2017-03-12
9103,30.68,2017-04-12


In [19]:
# função que junta os registros da estação e do reservatório onde as datas forem iguais e descarta os registros em que as datas não possuem um par
df_estacao_Resende_E_df_reservatorio_Funil = pd.merge(df_estacao_Resende_diaria_convertida, df_reservatorio_Funil_diario_convertido, how = 'inner', on = 'data')
df_estacao_Resende_E_df_reservatorio_Funil

,data,evap,inso,prec,temp,umid,vent,volu
0,1993-01-04,0.0,0.2,1.2,23.32,80.75,0.0,77.40
1,1993-02-04,1.5,0.1,0.0,21.92,93.75,0.0,78.06
2,1993-03-04,0.8,1.5,5.7,23.16,78.00,1266667.0,78.22
3,1993-04-04,1.3,8.2,0.0,23.96,76.75,1366667.0,78.22
4,1993-05-04,1.5,7.8,28.6,23.84,77.75,0.0,78.39
...,...,...,...,...,...,...,...,...
5887,2017-08-28,5.4,10.6,0.0,18.54,71.50,154332.0,26.73
5888,2017-09-30,4.3,0.0,23.8,19.12,97.25,0.0,14.81
5889,2017-11-14,8.9,12.2,0.0,22.06,55.00,0.0,26.38
5890,2017-11-17,9.5,9.0,0.0,24.16,68.75,0.0,24.77


In [20]:
# verificando a existência de duplicatas
df_estacao_Resende_E_df_reservatorio_Funil_sem_duplicados = df_estacao_Resende_E_df_reservatorio_Funil[df_estacao_Resende_E_df_reservatorio_Funil.duplicated()]
df_estacao_Resende_E_df_reservatorio_Funil_sem_duplicados

,data,evap,inso,prec,temp,umid,vent,volu


In [21]:
# visualizar os tipos de dados
df_estacao_Resende_E_df_reservatorio_Funil.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5892 entries, 0 to 5891
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   data    5892 non-null   datetime64[ns]
 1   evap    5892 non-null   float64       
 2   inso    5892 non-null   float64       
 3   prec    5892 non-null   float64       
 4   temp    5892 non-null   float64       
 5   umid    5892 non-null   float64       
 6   vent    5892 non-null   float64       
 7   volu    5892 non-null   float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 414.3 KB


In [22]:
# cálculo da perda de dados após o tratamento
# Estação de Resende (RJ) + Reservatório de Funil (RJ)
df_estacao_Resende_E_df_reservatorio_Funil_pct_perda = 100.0 - (len(df_estacao_Resende_E_df_reservatorio_Funil.index) * 100.0 / len(df_estacao_Resende_diaria.index))
print("Quantidade de perda de dados: " + str(round(df_estacao_Resende_E_df_reservatorio_Funil_pct_perda)) + "%")

Quantidade de perda de dados: 58%


In [23]:
# exportando dataframe unificado
df_estacao_Resende_E_df_reservatorio_Funil.to_csv('/content/drive/MyDrive/Colab Notebooks/TCC_PUC-Minas/Dados/df_estacao_reservatorio.csv', index=False)

In [24]:
# exportando dataframe unificado
df_estacao_Resende_E_df_reservatorio_Funil.to_excel('/content/drive/MyDrive/Colab Notebooks/TCC_PUC-Minas/Dados/df_estacao_reservatorio.xlsx', index=False)

In [25]:
# exportando dataframe unificado
df_estacao_Resende_E_df_reservatorio_Funil.to_pickle('/content/drive/MyDrive/Colab Notebooks/TCC_PUC-Minas/Dados/df_estacao_reservatorio.pkl')

### B - Gráficos

In [26]:
# LEGENDA: 
# prec = precipitação mm    | volu = volume útil %
# ------------------------------------------------------------------------------------------------
# usar uma das variáveis abaixo na linha do comentário => testar variável
# temp = temperatura do ar °C | umid = umidade relativa % | vent = velocidade do vento m/s
# evap = evaporação mm        | inso = insolação total h 

# agrupando a data para um período mensal
df_mensal = df_estacao_Resende_E_df_reservatorio_Funil.groupby(pd.Grouper(key='data', freq='M')).agg('mean')
df_mensal.dropna(inplace=True)
series_data = df_mensal.index.astype('string')
series_data_presente = df_mensal[df_mensal['prec'] > 0].index.astype('string')
series_data_ausente = df_mensal[df_mensal['prec'] == 0].index.astype('string')

# normalizando os valores
df_mensal_normal = (df_mensal-df_mensal.min())/(df_mensal.max()-df_mensal.min())

# criação da área do gráfico
fig = make_subplots()

# criação das barras com presença de precipitação/chuva
fig.add_trace(go.Bar(name='Presença', x=series_data_presente, y=df_mensal_normal[df_mensal_normal['prec'] > 0]['evap'], marker=dict(color='#3d85c6'))) # testar variável

# criação das barras com ausência de precipitação/chuva
fig.add_trace(go.Bar(name='Ausência', x=series_data_ausente, y=df_mensal_normal[df_mensal_normal['prec'] == 0]['evap'])) # testar variável

# criação das linha do volume útil
fig.add_trace(go.Line(name='Volume útil', x=series_data, y=df_mensal_normal['volu'], marker=dict(color='#0000ff')))

# título do gráfico
fig.update_layout(title="Presença e ausência da chuva no reservatório de Funil (RJ) em relação a evaporação", font=dict(family="Courier New, monospace", size=15, color="Purple"))

# exibição do gráfico
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




### Marina Micas Jardim


---

###### Trabalho de Conclusão de Curso apresentado ao Curso de Especialização em Inteligência Artificial e Aprendizado de Máquina, como requisito parcial à obtenção do título de Especialista.